# Risk Sensitivity Analysis
This notebook performs a sensitivity analysis on the assigned risk scores. The analysis is conducted in two parts: first by varying the severity properties, and then by perturbing the likelihood scores.

In [ ]:
# import necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from SALib.analyze import sobol
from SALib.sample import saltelli 
import pickle
import networkx as nx
import copy

from model_find_paths import connect_distribution_to_postnl

## Baseline risks

We first load the baseline risk values used in the study. These serve as the reference point for the adjustments made during the sensitivity analysis.

In [ ]:
df = pd.read_csv("../2.risk_analysis/input/risk_scores.csv") 

# drop last 3 rows 
df = df.drop(df.index[-3:])


In [4]:
df

,area_type,Sf,Sp,Ss,L1,L2,L3,L4,L5,Height
0,Motorways and major roads,4.0,3.0,1.0,3.0,2.0,1.0,1.0,3.0,30.0
1,Regional roads,3.0,3.0,2.0,4.0,3.0,1.0,3.0,2.0,30.0
2,Tracks and rural access roads,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,30.0
3,Living and residential streets,3.0,4.0,4.0,5.0,4.0,1.0,5.0,3.0,30.0
4,Pedestrian and cycling paths,3.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,30.0
5,Railways,4.0,3.0,2.0,5.0,2.0,2.0,1.0,4.0,30.0
6,Power lines,1.0,4.0,1.0,2.0,2.0,3.0,1.0,5.0,60.0
7,Power plants,1.0,4.0,2.0,4.0,3.0,2.0,3.0,5.0,60.0
8,Communication towers,1.0,3.0,1.0,4.0,5.0,1.0,2.0,4.0,60.0
9,High infrastructures,1.0,3.0,2.0,4.0,2.0,1.0,3.0,3.0,60.0


We will run both sensitivity analyses using the city of Breda as the test area.

In [ ]:
city = 'breda'
depot = ['Breda']

with open(f'/Users/cmartens/Documents/thesis_cf_martens/model/graph_creation/output/{city}.pkl', 'rb') as f:
    G = pickle.load(f)


# SA1: Sensitivity on severity properties

In [ ]:
def run_severity_sensitivity_analysis(G, df, alpha_values=[0.0, 0.5, 1.0], method="dijkstra"):
    """
    Run severity sensitivity analysis with the same metrics as energy sensitivity.
    
    Args:
        G: NetworkX graph
        df: DataFrame with risk scores
        alpha_values: List of alpha values to test
        method: Pathfinding method ('dijkstra' or 'astar')
    
    Returns:
        dict: Results organized by alpha value, containing DataFrames with metrics
    """
    
    # Area types and severity parameters for SA
    target_area_types = [
        "Meadows and open grass", 
        "Pedestrian and cycling paths", 
        "Regional roads", 
        "Rivers, canals and streams", 
        "Tracks and rural access roads"
    ]
    
    severity_cols = ["Sf", "Sp", "Ss"]
    severity_names = ["fatality", "property", "societal"]
    
    # First, calculate baseline results
    print("Calculating baseline results...")
    baseline_results = calculate_baseline_severity(G, df, alpha_values, method)
    
    all_results = {}
    
    for alpha in alpha_values:
        print(f"\n{'='*60}")
        print(f"ALPHA = {alpha}")
        print(f"{'='*60}")
        
        alpha_results = {}
        summary_data = []
        
        # Add baseline to summary
        baseline_alpha = baseline_results[baseline_results['alpha'] == alpha].iloc[0]
        baseline_summary = {
            'scenario': 'Baseline',
            'area_type': 'N/A',
            'parameter': 'N/A',
            'change_direction': 0,
            'alpha': alpha,
            'n_paths': baseline_alpha['n_paths'],
            'n_failed': baseline_alpha['n_failed'],
            'mean_length': baseline_alpha['mean_length'],
            'min_length': baseline_alpha['min_length'],
            'max_length': baseline_alpha['max_length'],
            'std_length': baseline_alpha['std_length'],
            'mean_risk': baseline_alpha['mean_risk'],
            'min_risk': baseline_alpha['min_risk'],
            'max_risk': baseline_alpha['max_risk'],
            'std_risk': baseline_alpha['std_risk'],
            'mean_energy': baseline_alpha['mean_energy'],
            'min_energy': baseline_alpha['min_energy'],
            'max_energy': baseline_alpha['max_energy'],
            'std_energy': baseline_alpha['std_energy'],
            'mean_turns': baseline_alpha['mean_turns'],
            'min_turns': baseline_alpha['min_turns'],
            'max_turns': baseline_alpha['max_turns'],
            'std_turns': baseline_alpha['std_turns'],
            'mean_height_changes': baseline_alpha['mean_height_changes'],
            'min_height_changes': baseline_alpha['min_height_changes'],
            'max_height_changes': baseline_alpha['max_height_changes'],
            'std_height_changes': baseline_alpha['std_height_changes'],
            'n_unique_etypes': baseline_alpha['n_unique_etypes']
        }
        summary_data.append(baseline_summary)
        
        # Run sensitivity scenarios
        for area in target_area_types:
            for i, severity_col in enumerate(severity_cols):
                severity_name = severity_names[i]
                for delta in [-1, +1]:
                    scenario_name = f"{area} - {severity_name} {'+' if delta > 0 else ''}{delta}"
                    print(f"\nRunning scenario: {scenario_name}")
                    
                    # Modify severity scores
                    df_mod = df.copy()
                    affected_rows = df_mod["area_type"] == area
                    df_mod.loc[affected_rows, severity_col] = (
                        df_mod.loc[affected_rows, severity_col] + delta
                    ).clip(lower=1, upper=4)
                    
                    # Recalculate risks
                    df_mod = recalculate_risks(df_mod)
                    
                    # Apply to graph
                    apply_risks_to_graph(G, df_mod)
                    
                    # Run pathfinding
                    connected, not_connected, metrics_df = connect_distribution_to_postnl(
                        G, alpha=alpha, method=method
                    )
                    
                    # Store raw results
                    alpha_results[scenario_name] = {
                        'connected': connected,
                        'not_connected': not_connected,
                        'metrics_df': metrics_df
                    }
                    
                    # Calculate summary statistics
                    if not metrics_df.empty:
                        # Collect edge types
                        etypes = []
                        for _, _, _, _, etype_array in connected:
                            etypes.extend(etype_array)
                        
                        summary = {
                            'scenario': scenario_name,
                            'area_type': area,
                            'parameter': severity_name,
                            'change_direction': delta,
                            'alpha': alpha,
                            'n_paths': len(metrics_df),
                            'n_failed': len(not_connected),
                            # Length statistics
                            'mean_length': metrics_df['length'].mean(),
                            'min_length': metrics_df['length'].min(),
                            'max_length': metrics_df['length'].max(),
                            'std_length': metrics_df['length'].std(),
                            # Risk statistics
                            'mean_risk': metrics_df['risk'].mean(),
                            'min_risk': metrics_df['risk'].min(),
                            'max_risk': metrics_df['risk'].max(),
                            'std_risk': metrics_df['risk'].std(),
                            # Energy statistics
                            'mean_energy': metrics_df['energy'].mean(),
                            'min_energy': metrics_df['energy'].min(),
                            'max_energy': metrics_df['energy'].max(),
                            'std_energy': metrics_df['energy'].std(),
                            # Turns statistics
                            'mean_turns': metrics_df['turns'].mean(),
                            'min_turns': metrics_df['turns'].min(),
                            'max_turns': metrics_df['turns'].max(),
                            'std_turns': metrics_df['turns'].std(),
                            # Height changes statistics
                            'mean_height_changes': metrics_df['height_changes'].mean(),
                            'min_height_changes': metrics_df['height_changes'].min(),
                            'max_height_changes': metrics_df['height_changes'].max(),
                            'std_height_changes': metrics_df['height_changes'].std(),
                            # Unique edge types
                            'n_unique_etypes': len(set(etypes))
                        }
                    else:
                        # No successful paths
                        summary = {
                            'scenario': scenario_name,
                            'area_type': area,
                            'parameter': severity_name,
                            'change_direction': delta,
                            'alpha': alpha,
                            'n_paths': 0,
                            'n_failed': len(not_connected),
                            **{col: np.nan for col in [
                                'mean_length', 'min_length', 'max_length', 'std_length',
                                'mean_risk', 'min_risk', 'max_risk', 'std_risk',
                                'mean_energy', 'min_energy', 'max_energy', 'std_energy',
                                'mean_turns', 'min_turns', 'max_turns', 'std_turns',
                                'mean_height_changes', 'min_height_changes', 'max_height_changes', 'std_height_changes'
                            ]},
                            'n_unique_etypes': 0
                        }
                    
                    summary_data.append(summary)
        
        # Create summary DataFrame for this alpha
        alpha_results['summary_df'] = pd.DataFrame(summary_data)
        all_results[f'alpha_{alpha}'] = alpha_results
        
        # Print summary table for this alpha
        print(f"\nSummary for alpha = {alpha}:")
        print(alpha_results['summary_df'][['scenario', 'n_paths', 'mean_length', 'mean_risk', 'mean_energy']].head(10).to_string(index=False))
    
    return all_results, baseline_results

def calculate_baseline_severity(G, df, alpha_values, method):
    """Calculate baseline results for comparison"""
    baseline_results = []
    
    for alpha in alpha_values:
        # Apply original risk calculations
        df_baseline = recalculate_risks(df.copy())
        apply_risks_to_graph(G, df_baseline)
        
        # Run pathfinding
        connected, not_connected, metrics_df = connect_distribution_to_postnl(
            G, alpha=alpha, method=method
        )
        
        if not metrics_df.empty:
            # Collect edge types
            etypes = []
            for _, _, _, _, etype_array in connected:
                etypes.extend(etype_array)
            
            baseline_results.append({
                'alpha': alpha,
                'n_paths': len(metrics_df),
                'n_failed': len(not_connected),
                'mean_length': metrics_df['length'].mean(),
                'min_length': metrics_df['length'].min(),
                'max_length': metrics_df['length'].max(),
                'std_length': metrics_df['length'].std(),
                'mean_risk': metrics_df['risk'].mean(),
                'min_risk': metrics_df['risk'].min(),
                'max_risk': metrics_df['risk'].max(),
                'std_risk': metrics_df['risk'].std(),
                'mean_energy': metrics_df['energy'].mean(),
                'min_energy': metrics_df['energy'].min(),
                'max_energy': metrics_df['energy'].max(),
                'std_energy': metrics_df['energy'].std(),
                'mean_turns': metrics_df['turns'].mean(),
                'min_turns': metrics_df['turns'].min(),
                'max_turns': metrics_df['turns'].max(),
                'std_turns': metrics_df['turns'].std(),
                'mean_height_changes': metrics_df['height_changes'].mean(),
                'min_height_changes': metrics_df['height_changes'].min(),
                'max_height_changes': metrics_df['height_changes'].max(),
                'std_height_changes': metrics_df['height_changes'].std(),
                'n_unique_etypes': len(set(etypes))
            })
    
    return pd.DataFrame(baseline_results)

def recalculate_risks(df):
    """Recalculate risk scores based on severity values"""
    n_factors = 5
    R_f_total, R_p_total = 0, 0
    
    for j in range(1, n_factors + 1):
        df[f"R_if_{j}"] = df["Sf"] * df[f"L{j}"]
        df[f"R_ip_{j}"] = df["Sp"] * df[f"L{j}"]
        R_f_total += df[f"R_if_{j}"]
        R_p_total += df[f"R_ip_{j}"]
    
    df["R_f"] = R_f_total
    df["R_p"] = R_p_total
    df["R_s"] = df["Ss"]
    
    # Normalization
    df["R_f_norm"] = (df["R_f"] - df["R_f"].min()) / (df["R_f"].max() - df["R_f"].min())
    df["R_p_norm"] = (df["R_p"] - df["R_p"].min()) / (df["R_p"].max() - df["R_p"].min())
    df["R_s_norm"] = (df["R_s"] - df["R_s"].min()) / (df["R_s"].max() - df["R_s"].min())
    
    # Weighted risk score
    df["risk"] = (
        0.4 * df["R_f_norm"] + 
        0.3 * df["R_p_norm"] + 
        0.3 * df["R_s_norm"]
    )
    
    return df

def apply_risks_to_graph(G, df):
    """Apply risk scores from dataframe to graph edges"""
    etype_to_risk = dict(zip(df["area_type"], df["risk"]))
    for u, v, data in G.edges(data=True):
        etype = data.get("etype")
        if etype in etype_to_risk:
            data["risk"] = etype_to_risk[etype]
        if etype == "postnl_connector":
            data["risk"] = 0.0

def compare_scenarios_to_baseline_severity(results_dict, alpha, metric='mean_risk'):
    """Compare all scenarios to baseline for severity analysis"""
    summary_df = results_dict[f'alpha_{alpha}']['summary_df']
    
    # Get baseline value
    base_value = summary_df[summary_df['scenario'] == 'Baseline'][metric].values[0]
    
    # Calculate differences
    comparison = summary_df.copy()
    comparison[f'{metric}_diff'] = comparison[metric] - base_value
    comparison[f'{metric}_pct_change'] = ((comparison[metric] - base_value) / base_value * 100)
    
    return comparison[['scenario', 'area_type', 'parameter', 'change_direction', metric, f'{metric}_diff', f'{metric}_pct_change']]

In [9]:
# Run the analysis
results, baseline_df = run_severity_sensitivity_analysis(G, df, alpha_values=[0, 0.5, 1], method="dijkstra")

Calculating baseline results...
Distribution points: 1
PostNL points: 64
Alpha: 0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m
Connected: 157648 → 1 | Length: 5411.2 m
Connected: 157648 → 2 | Length: 7091.7 m
Connected: 157648 → 3 | Length: 4856.1 m
Connected: 157648 → 4 | Length: 3481.7 m
Connected: 157648 → 5 | Length: 3151.6 m
Connected: 157648 → 6 | Length: 4250.9 m
Connected: 157648 → 7 | Length: 2861.1 m
Connected: 157648 → 8 | Length: 2643.8 m
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m
Connected: 157648 → 11 | Length: 5987.9 m
Connected: 157648 → 12 | Length: 3133.2 m
Connected: 157648 → 13 | Length: 5332.2 m
Connected: 157648 → 14 | Length: 6446.6 m
Connected: 157648 → 15 | Length: 6200.1 m
Connected: 157648 → 16 | Length: 4979.6 m
Connected: 157648 → 17 | Length: 3454.6 m
Connected: 157648 → 18 | Length: 3149.3 m
Connected: 157648 → 19 | Length: 2876.2 m
Connected: 157648 → 20 | Length: 4510.6 m
Connected: 157648 → 21 | Length: 5548.9 m
Connected: 15764

In [132]:
def create_severity_sa_summary(results):
    """
    Create summary DataFrames from severity sensitivity analysis results with all metrics.
    """
    # 1. Combined DataFrame with all results
    all_data = []
    for key, value in results.items():
        if 'alpha_' in str(key):
            alpha = float(str(key).replace('alpha_', ''))
            if isinstance(value, dict) and 'summary_df' in value:
                df = value['summary_df'].copy()
                all_data.append(df)
    
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Remove std columns for cleaner view
    std_cols = [col for col in combined_df.columns if 'std_' in col]
    combined_df_clean = combined_df.drop(columns=std_cols)
    
    # 2. Baseline comparison
    baseline_comparison = []
    for alpha in [0.0, 0.5, 1.0]:
        alpha_df = combined_df[combined_df['alpha'] == alpha]
        baseline = alpha_df[alpha_df['scenario'] == 'Baseline'].iloc[0]
        
        for _, row in alpha_df.iterrows():
            if row['scenario'] != 'Baseline':
                comparison = {
                    'alpha': alpha,
                    'scenario': row['scenario'],
                    'area_type': row['area_type'],
                    'parameter': row['parameter'],
                    'change_direction': row['change_direction'],
                    # Percentage changes for all metrics
                    'length_change_%': ((row['mean_length'] - baseline['mean_length']) / baseline['mean_length'] * 100),
                    'risk_change_%': ((row['mean_risk'] - baseline['mean_risk']) / baseline['mean_risk'] * 100),
                    'energy_change_%': ((row['mean_energy'] - baseline['mean_energy']) / baseline['mean_energy'] * 100),
                    'turns_change_%': ((row['mean_turns'] - baseline['mean_turns']) / baseline['mean_turns'] * 100) if baseline['mean_turns'] > 0 else 0,
                    'height_changes_change_%': ((row['mean_height_changes'] - baseline['mean_height_changes']) / baseline['mean_height_changes'] * 100) if baseline['mean_height_changes'] > 0 else 0,
                    # Absolute values
                    'mean_length': row['mean_length'],
                    'mean_risk': row['mean_risk'],
                    'mean_energy': row['mean_energy'],
                    'mean_turns': row['mean_turns'],
                    'mean_height_changes': row['mean_height_changes']
                }
                baseline_comparison.append(comparison)
    
    comparison_df = pd.DataFrame(baseline_comparison)
    
    # 3. Summary by area type and parameter (including all metrics)
    area_param_summary = comparison_df.groupby(['alpha', 'area_type', 'parameter']).agg({
        'risk_change_%': ['mean', 'min', 'max'],
        'length_change_%': ['mean', 'min', 'max'],
        'energy_change_%': ['mean', 'min', 'max'],
        'turns_change_%': ['mean', 'min', 'max'],
        'height_changes_change_%': ['mean', 'min', 'max']
    }).round(2)
    
    # 4. Summary by area type (averaging across all parameters)
    area_summary = comparison_df.groupby(['alpha', 'area_type']).agg({
        'risk_change_%': ['mean', 'std', 'min', 'max'],
        'length_change_%': ['mean', 'std', 'min', 'max'],
        'energy_change_%': ['mean', 'std', 'min', 'max'],
        'turns_change_%': ['mean', 'std', 'min', 'max'],
        'height_changes_change_%': ['mean', 'std', 'min', 'max']
    }).round(2)
    
    # 5. Most sensitive scenarios
    most_sensitive = comparison_df.nlargest(10, 'risk_change_%')[
        ['alpha', 'area_type', 'parameter', 'change_direction', 
         'risk_change_%', 'length_change_%', 'energy_change_%', 
         'turns_change_%', 'height_changes_change_%']
    ]
    
    # 6. Summary statistics per alpha
    alpha_summary = []
    for alpha in [0.0, 0.5, 1.0]:
        alpha_data = comparison_df[comparison_df['alpha'] == alpha]
        summary = {
            'alpha': alpha,
            'avg_risk_change_%': alpha_data['risk_change_%'].mean(),
            'avg_length_change_%': alpha_data['length_change_%'].mean(),
            'avg_energy_change_%': alpha_data['energy_change_%'].mean(),
            'avg_turns_change_%': alpha_data['turns_change_%'].mean(),
            'avg_height_changes_%': alpha_data['height_changes_change_%'].mean(),
        }
        alpha_summary.append(summary)
    
    alpha_summary_df = pd.DataFrame(alpha_summary)
    
    return {
        'combined': combined_df_clean,
        'comparison': comparison_df,
        'area_param_summary': area_param_summary,
        'area_summary': area_summary,
        'most_sensitive': most_sensitive,
        'alpha_summary': alpha_summary_df
    }

In [133]:
# Cell 2: Create summaries
summaries = create_severity_sa_summary(results)

# Display alpha summary
print("=== SUMMARY BY ALPHA ===")
summaries['alpha_summary']

=== SUMMARY BY ALPHA ===


,alpha,avg_risk_change_%,avg_length_change_%,avg_energy_change_%,avg_turns_change_%,avg_height_changes_%
0,0.0,0.386979,0.000000,0.000000,0.000000,0.000000
1,0.5,4.449714,-0.223214,0.096506,1.680397,0.733163
2,1.0,4.697242,-1.894458,-1.361143,-1.091582,3.013923


In [134]:
summaries

{'combined':                                        scenario  \
 0                                      Baseline   
 1          Meadows and open grass - fatality -1   
 2          Meadows and open grass - fatality +1   
 3          Meadows and open grass - property -1   
 4          Meadows and open grass - property +1   
 ..                                          ...   
 88  Tracks and rural access roads - fatality +1   
 89  Tracks and rural access roads - property -1   
 90  Tracks and rural access roads - property +1   
 91  Tracks and rural access roads - societal -1   
 92  Tracks and rural access roads - societal +1   
 
                         area_type parameter  change_direction  alpha  n_paths  \
 0                             N/A       N/A                 0    0.0     63.0   
 1          Meadows and open grass  fatality                -1    0.0     63.0   
 2          Meadows and open grass  fatality                 1    0.0     63.0   
 3          Meadows and open grass

In [136]:
# Cell 8: Create summary for specific alpha
alpha_val = 0.0 # Change this to analyze different alpha
alpha_data = summaries['comparison'][summaries['comparison']['alpha'] == alpha_val]

# Group by area and parameter
summary_table = alpha_data.pivot_table(
    values=['risk_change_%', 'length_change_%', 'energy_change_%', 'turns_change_%', 'height_changes_change_%'],
    index='area_type',
    columns='parameter',
    aggfunc='mean'
).round(2)

print(f"=== DETAILED SUMMARY FOR α = {alpha_val} ===")
summary_table

=== DETAILED SUMMARY FOR α = 0.0 ===


energy_change_%                    \
parameter                            fatality property societal   
area_type                                                         
Meadows and open grass                    0.0      0.0      0.0   
Pedestrian and cycling paths              0.0      0.0      0.0   
Regional roads                            0.0      0.0      0.0   
Rivers, canals and streams                0.0      0.0      0.0   
Tracks and rural access roads             0.0      0.0      0.0   

                              height_changes_change_%                    \
parameter                                    fatality property societal   
area_type                                                                 
Meadows and open grass                            0.0      0.0      0.0   
Pedestrian and cycling paths                      0.0      0.0      0.0   
Regional roads                                    0.0      0.0      0.0   
Rivers, canals and streams                        0.0      0.0      0.0   
Tracks and rural access roads                     0.0      0.0      0.0   

                              length_change_%                   risk_change_%  \
parameter                            fatality property societal      fatality   
area_type                                                                       
Meadows and open grass                    0.0      0.0      0.0          0.49   
Pedestrian and cycling paths              0.0      0.0      0.0          0.00   
Regional roads                            0.0      0.0      0.0          0.00   
Rivers, canals and streams                0.0      0.0      0.0          0.22   
Tracks and rural access roads             0.0      0.0      0.0          0.00   

                                                turns_change_%           \
parameter                     property societal       fatality property   
area_type                                                                 
Meadows and open grass            0.29     1.08            0.0      0.0   
Pedestrian and cycling paths      0.00    -0.00            0.0      0.0   
Regional roads                    0.00     0.00            0.0      0.0   
Rivers, canals and streams        0.13     0.48            0.0      0.0   
Tracks and rural access roads     0.00     3.13            0.0      0.0   

                                        
parameter                     societal  
area_type                               
Meadows and open grass             0.0  
Pedestrian and cycling paths       0.0  
Regional roads                     0.0  
Rivers, canals and streams         0.0  
Tracks and rural access roads      0.0

In [137]:
# Cell 8: Create summary for specific alpha
alpha_val = 0.5  # Change this to analyze different alpha
alpha_data = summaries['comparison'][summaries['comparison']['alpha'] == alpha_val]

# Group by area and parameter
summary_table = alpha_data.pivot_table(
    values=['risk_change_%', 'length_change_%', 'energy_change_%', 'turns_change_%', 'height_changes_change_%'],
    index='area_type',
    columns='parameter',
    aggfunc='mean'
).round(2)

print(f"=== DETAILED SUMMARY FOR α = {alpha_val} ===")
summary_table

=== DETAILED SUMMARY FOR α = 0.5 ===


energy_change_%                    \
parameter                            fatality property societal   
area_type                                                         
Meadows and open grass                   0.80     0.31     1.12   
Pedestrian and cycling paths            -0.31    -0.01    -0.36   
Regional roads                           0.46     0.59     0.48   
Rivers, canals and streams              -1.48    -0.24    -2.56   
Tracks and rural access roads            0.56     1.04     1.07   

                              height_changes_change_%                    \
parameter                                    fatality property societal   
area_type                                                                 
Meadows and open grass                           0.77     0.77     0.26   
Pedestrian and cycling paths                     0.77     0.51     0.77   
Regional roads                                   0.77     0.77     0.77   
Rivers, canals and streams                       0.51     0.51     1.53   
Tracks and rural access roads                    1.02     0.51     0.77   

                              length_change_%                   risk_change_%  \
parameter                            fatality property societal      fatality   
area_type                                                                       
Meadows and open grass                   0.81     0.28     1.17          6.37   
Pedestrian and cycling paths            -0.37    -0.03    -0.42         -0.16   
Regional roads                           0.44     0.59     0.47         -0.63   
Rivers, canals and streams              -1.59    -0.26    -2.78         13.21   
Tracks and rural access roads           -0.90    -0.35    -0.41         -1.24   

                                                turns_change_%           \
parameter                     property societal       fatality property   
area_type                                                                 
Meadows and open grass            4.39    10.70           1.85     1.25   
Pedestrian and cycling paths     -0.09    -0.34          -0.58    -0.37   
Regional roads                   -0.42    -0.79           1.01     0.82   
Rivers, canals and streams        8.36    22.13           4.83     5.47   
Tracks and rural access roads    -0.65     5.91           0.72     0.74   

                                        
parameter                     societal  
area_type                               
Meadows and open grass            2.70  
Pedestrian and cycling paths     -0.62  
Regional roads                    0.89  
Rivers, canals and streams        6.32  
Tracks and rural access roads     0.19

In [138]:
# Cell 8: Create summary for specific alpha
alpha_val = 1 # Change this to analyze different alpha
alpha_data = summaries['comparison'][summaries['comparison']['alpha'] == alpha_val]

# Group by area and parameter
summary_table = alpha_data.pivot_table(
    values=['risk_change_%', 'length_change_%', 'energy_change_%', 'turns_change_%', 'height_changes_change_%'],
    index='area_type',
    columns='parameter',
    aggfunc='mean'
).round(2)

print(f"=== DETAILED SUMMARY FOR α = {alpha_val} ===")
summary_table

=== DETAILED SUMMARY FOR α = 1 ===


energy_change_%                    \
parameter                            fatality property societal   
area_type                                                         
Meadows and open grass                  -2.77    -2.33    -2.49   
Pedestrian and cycling paths            -0.25    -0.30    -0.39   
Regional roads                          -0.85    -0.01    -0.90   
Rivers, canals and streams              -4.12    -3.74    -4.39   
Tracks and rural access roads            0.90     1.22     0.01   

                              height_changes_change_%                    \
parameter                                    fatality property societal   
area_type                                                                 
Meadows and open grass                          -0.74    -0.74    -0.25   
Pedestrian and cycling paths                    -0.25     0.25     0.00   
Regional roads                                   0.74     0.74     0.74   
Rivers, canals and streams                       6.39     6.39    26.29   
Tracks and rural access roads                    2.70     0.00     2.95   

                              length_change_%                   risk_change_%  \
parameter                            fatality property societal      fatality   
area_type                                                                       
Meadows and open grass                  -2.93    -2.45    -2.66          6.97   
Pedestrian and cycling paths            -0.26    -0.33    -0.43         -0.30   
Regional roads                          -1.07    -0.15    -1.15         -0.45   
Rivers, canals and streams              -4.67    -4.26    -6.15         13.75   
Tracks and rural access roads           -0.46     0.00    -1.43         -1.45   

                                                turns_change_%           \
parameter                     property societal       fatality property   
area_type                                                                 
Meadows and open grass            4.76    11.45          -4.08    -3.73   
Pedestrian and cycling paths     -0.08    -0.56           0.69    -0.21   
Regional roads                   -0.14    -0.66          -0.52     0.13   
Rivers, canals and streams        9.03    22.80          -2.55    -3.33   
Tracks and rural access roads    -0.65     5.99           1.15     1.11   

                                        
parameter                     societal  
area_type                               
Meadows and open grass           -3.03  
Pedestrian and cycling paths      0.49  
Regional roads                   -0.57  
Rivers, canals and streams       -1.17  
Tracks and rural access roads    -0.76

In [139]:
# save dictionary to pickle file
import pickle
with open('/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/severity_sensitivity_analysis_results.pkl', 'wb') as f:
    pickle.dump(results, f)


# Sensitivity Analysis on Likelihood Scores

In [ ]:
def run_likelihood_sensitivity_analysis(G, df, alpha_values=[0.0, 0.5, 1.0], method="dijkstra"):
    """
    Run likelihood sensitivity analysis with the same metrics as severity sensitivity.
    
    Args:
        G: NetworkX graph
        df: DataFrame with risk scores
        alpha_values: List of alpha values to test
        method: Pathfinding method ('dijkstra' or 'astar')
    
    Returns:
        dict: Results organized by alpha value, containing DataFrames with metrics
    """
    
    # Area types and likelihood parameters for SA
    target_area_types = [
        "Meadows and open grass", 
        "Pedestrian and cycling paths", 
        "Regional roads", 
        "Rivers, canals and streams", 
        "Tracks and rural access roads"
    ]
    
    likelihood_cols = ["L1", "L2", "L3", "L4", "L5"]
    likelihood_names = ["Obstacles", "Interference", "Communication", "Navigational", "Electrical"]  # Based on your risk factors
    
    # First, calculate baseline results
    print("Calculating baseline results...")
    baseline_results = calculate_baseline_likelihood(G, df, alpha_values, method)
    
    all_results = {}
    
    for alpha in alpha_values:
        print(f"\n{'='*60}")
        print(f"ALPHA = {alpha}")
        print(f"{'='*60}")
        
        alpha_results = {}
        summary_data = []
        
        # Add baseline to summary
        baseline_alpha = baseline_results[baseline_results['alpha'] == alpha].iloc[0]
        baseline_summary = {
            'scenario': 'Baseline',
            'area_type': 'N/A',
            'parameter': 'N/A',
            'change_direction': 0,
            'alpha': alpha,
            'n_paths': baseline_alpha['n_paths'],
            'n_failed': baseline_alpha['n_failed'],
            'mean_length': baseline_alpha['mean_length'],
            'min_length': baseline_alpha['min_length'],
            'max_length': baseline_alpha['max_length'],
            'std_length': baseline_alpha['std_length'],
            'mean_risk': baseline_alpha['mean_risk'],
            'min_risk': baseline_alpha['min_risk'],
            'max_risk': baseline_alpha['max_risk'],
            'std_risk': baseline_alpha['std_risk'],
            'mean_energy': baseline_alpha['mean_energy'],
            'min_energy': baseline_alpha['min_energy'],
            'max_energy': baseline_alpha['max_energy'],
            'std_energy': baseline_alpha['std_energy'],
            'mean_turns': baseline_alpha['mean_turns'],
            'min_turns': baseline_alpha['min_turns'],
            'max_turns': baseline_alpha['max_turns'],
            'std_turns': baseline_alpha['std_turns'],
            'mean_height_changes': baseline_alpha['mean_height_changes'],
            'min_height_changes': baseline_alpha['min_height_changes'],
            'max_height_changes': baseline_alpha['max_height_changes'],
            'std_height_changes': baseline_alpha['std_height_changes'],
            'n_unique_etypes': baseline_alpha['n_unique_etypes']
        }
        summary_data.append(baseline_summary)
        
        # Run sensitivity scenarios
        for area in target_area_types:
            for i, factor in enumerate(likelihood_cols):
                factor_name = likelihood_names[i]
                for delta in [-1, +1]:
                    scenario_name = f"{area} - {factor_name} {'+' if delta > 0 else ''}{delta}"
                    print(f"\nRunning scenario: {scenario_name}")
                    
                    # Modify likelihood scores
                    df_mod = df.copy()
                    affected_rows = df_mod["area_type"] == area
                    df_mod.loc[affected_rows, factor] = (
                        df_mod.loc[affected_rows, factor] + delta
                    ).clip(lower=1, upper=5)  # Likelihood scale is 1-5
                    
                    # Recalculate risks
                    df_mod = recalculate_risks(df_mod)
                    
                    # Apply to graph
                    apply_risks_to_graph(G, df_mod)
                    
                    # Run pathfinding
                    connected, not_connected, metrics_df = connect_distribution_to_postnl(
                        G, alpha=alpha, method=method
                    )
                    
                    # Store raw results
                    alpha_results[scenario_name] = {
                        'connected': connected,
                        'not_connected': not_connected,
                        'metrics_df': metrics_df
                    }
                    
                    # Calculate summary statistics
                    if not metrics_df.empty:
                        # Collect edge types
                        etypes = []
                        for _, _, _, _, etype_array in connected:
                            etypes.extend(etype_array)
                        
                        summary = {
                            'scenario': scenario_name,
                            'area_type': area,
                            'parameter': factor_name,
                            'likelihood_factor': factor,
                            'change_direction': delta,
                            'alpha': alpha,
                            'n_paths': len(metrics_df),
                            'n_failed': len(not_connected),
                            # Length statistics
                            'mean_length': metrics_df['length'].mean(),
                            'min_length': metrics_df['length'].min(),
                            'max_length': metrics_df['length'].max(),
                            'std_length': metrics_df['length'].std(),
                            # Risk statistics
                            'mean_risk': metrics_df['risk'].mean(),
                            'min_risk': metrics_df['risk'].min(),
                            'max_risk': metrics_df['risk'].max(),
                            'std_risk': metrics_df['risk'].std(),
                            # Energy statistics
                            'mean_energy': metrics_df['energy'].mean(),
                            'min_energy': metrics_df['energy'].min(),
                            'max_energy': metrics_df['energy'].max(),
                            'std_energy': metrics_df['energy'].std(),
                            # Turns statistics
                            'mean_turns': metrics_df['turns'].mean(),
                            'min_turns': metrics_df['turns'].min(),
                            'max_turns': metrics_df['turns'].max(),
                            'std_turns': metrics_df['turns'].std(),
                            # Height changes statistics
                            'mean_height_changes': metrics_df['height_changes'].mean(),
                            'min_height_changes': metrics_df['height_changes'].min(),
                            'max_height_changes': metrics_df['height_changes'].max(),
                            'std_height_changes': metrics_df['height_changes'].std(),
                            # Unique edge types
                            'n_unique_etypes': len(set(etypes))
                        }
                    else:
                        # No successful paths
                        summary = {
                            'scenario': scenario_name,
                            'area_type': area,
                            'parameter': factor_name,
                            'likelihood_factor': factor,
                            'change_direction': delta,
                            'alpha': alpha,
                            'n_paths': 0,
                            'n_failed': len(not_connected),
                            **{col: np.nan for col in [
                                'mean_length', 'min_length', 'max_length', 'std_length',
                                'mean_risk', 'min_risk', 'max_risk', 'std_risk',
                                'mean_energy', 'min_energy', 'max_energy', 'std_energy',
                                'mean_turns', 'min_turns', 'max_turns', 'std_turns',
                                'mean_height_changes', 'min_height_changes', 'max_height_changes', 'std_height_changes'
                            ]},
                            'n_unique_etypes': 0
                        }
                    
                    summary_data.append(summary)
        
        # Create summary DataFrame for this alpha
        alpha_results['summary_df'] = pd.DataFrame(summary_data)
        all_results[f'alpha_{alpha}'] = alpha_results
        
        # Print summary table for this alpha
        print(f"\nSummary for alpha = {alpha}:")
        print(alpha_results['summary_df'][['scenario', 'n_paths', 'mean_length', 'mean_risk', 'mean_energy']].head(10).to_string(index=False))
    
    return all_results, baseline_results

def calculate_baseline_likelihood(G, df, alpha_values, method):
    """Calculate baseline results for comparison"""
    baseline_results = []
    
    for alpha in alpha_values:
        # Apply original risk calculations
        df_baseline = recalculate_risks(df.copy())
        apply_risks_to_graph(G, df_baseline)
        
        # Run pathfinding
        connected, not_connected, metrics_df = connect_distribution_to_postnl(
            G, alpha=alpha, method=method
        )
        
        if not metrics_df.empty:
            # Collect edge types
            etypes = []
            for _, _, _, _, etype_array in connected:
                etypes.extend(etype_array)
            
            baseline_results.append({
                'alpha': alpha,
                'n_paths': len(metrics_df),
                'n_failed': len(not_connected),
                'mean_length': metrics_df['length'].mean(),
                'min_length': metrics_df['length'].min(),
                'max_length': metrics_df['length'].max(),
                'std_length': metrics_df['length'].std(),
                'mean_risk': metrics_df['risk'].mean(),
                'min_risk': metrics_df['risk'].min(),
                'max_risk': metrics_df['risk'].max(),
                'std_risk': metrics_df['risk'].std(),
                'mean_energy': metrics_df['energy'].mean(),
                'min_energy': metrics_df['energy'].min(),
                'max_energy': metrics_df['energy'].max(),
                'std_energy': metrics_df['energy'].std(),
                'mean_turns': metrics_df['turns'].mean(),
                'min_turns': metrics_df['turns'].min(),
                'max_turns': metrics_df['turns'].max(),
                'std_turns': metrics_df['turns'].std(),
                'mean_height_changes': metrics_df['height_changes'].mean(),
                'min_height_changes': metrics_df['height_changes'].min(),
                'max_height_changes': metrics_df['height_changes'].max(),
                'std_height_changes': metrics_df['height_changes'].std(),
                'n_unique_etypes': len(set(etypes))
            })
    
    return pd.DataFrame(baseline_results)

def recalculate_risks(df):
    """Recalculate risk scores based on current likelihood and severity values"""
    n_factors = 5
    R_f_total, R_p_total = 0, 0
    
    for j in range(1, n_factors + 1):
        df[f"R_if_{j}"] = df["Sf"] * df[f"L{j}"]
        df[f"R_ip_{j}"] = df["Sp"] * df[f"L{j}"]
        R_f_total += df[f"R_if_{j}"]
        R_p_total += df[f"R_ip_{j}"]
    
    df["R_f"] = R_f_total
    df["R_p"] = R_p_total
    df["R_s"] = df["Ss"]
    
    # Normalization
    df["R_f_norm"] = (df["R_f"] - df["R_f"].min()) / (df["R_f"].max() - df["R_f"].min())
    df["R_p_norm"] = (df["R_p"] - df["R_p"].min()) / (df["R_p"].max() - df["R_p"].min())
    df["R_s_norm"] = (df["R_s"] - df["R_s"].min()) / (df["R_s"].max() - df["R_s"].min())
    
    # Weighted risk score
    df["risk"] = (
        0.4 * df["R_f_norm"] + 
        0.3 * df["R_p_norm"] + 
        0.3 * df["R_s_norm"]
    )
    
    return df

def apply_risks_to_graph(G, df):
    """Apply risk scores from dataframe to graph edges"""
    etype_to_risk = dict(zip(df["area_type"], df["risk"]))
    for u, v, data in G.edges(data=True):
        etype = data.get("etype")
        if etype in etype_to_risk:
            data["risk"] = etype_to_risk[etype]
        if etype == "postnl_connector":
            data["risk"] = 0.0


In [38]:
likelihood_results, likelihood_baseline = run_likelihood_sensitivity_analysis(
        G, df, alpha_values=[0, 0.5, 1], method="dijkstra"
    )

Calculating baseline results...
Distribution points: 1
PostNL points: 64
Alpha: 0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m
Connected: 157648 → 1 | Length: 5411.2 m
Connected: 157648 → 2 | Length: 7091.7 m
Connected: 157648 → 3 | Length: 4856.1 m
Connected: 157648 → 4 | Length: 3481.7 m
Connected: 157648 → 5 | Length: 3151.6 m
Connected: 157648 → 6 | Length: 4250.9 m
Connected: 157648 → 7 | Length: 2861.1 m
Connected: 157648 → 8 | Length: 2643.8 m
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m
Connected: 157648 → 11 | Length: 5987.9 m
Connected: 157648 → 12 | Length: 3133.2 m
Connected: 157648 → 13 | Length: 5332.2 m
Connected: 157648 → 14 | Length: 6446.6 m
Connected: 157648 → 15 | Length: 6200.1 m
Connected: 157648 → 16 | Length: 4979.6 m
Connected: 157648 → 17 | Length: 3454.6 m
Connected: 157648 → 18 | Length: 3149.3 m
Connected: 157648 → 19 | Length: 2876.2 m
Connected: 157648 → 20 | Length: 4510.6 m
Connected: 157648 → 21 | Length: 5548.9 m
Connected: 15764

In [103]:
def create_likelihood_sa_summary(results):
    """Create summary DataFrames from likelihood sensitivity analysis results."""
    
    # 1. Combined DataFrame with all results
    all_data = []
    for key, value in results.items():
        if 'alpha_' in str(key):
            if isinstance(value, dict) and 'summary_df' in value:
                df = value['summary_df'].copy()
                all_data.append(df)
    
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Remove std columns
    std_cols = [col for col in combined_df.columns if 'std_' in col]
    combined_df_clean = combined_df.drop(columns=std_cols)
    
    # 2. Baseline comparison
    baseline_comparison = []
    for alpha in [0.0, 0.5, 1.0]:
        alpha_df = combined_df[combined_df['alpha'] == alpha]
        baseline = alpha_df[alpha_df['scenario'] == 'Baseline'].iloc[0]
        
        for _, row in alpha_df.iterrows():
            if row['scenario'] != 'Baseline':
                comparison = {
                    'alpha': alpha,
                    'scenario': row['scenario'],
                    'area_type': row['area_type'],
                    'parameter': row['parameter'],
                    'change_direction': row['change_direction'],
                    'length_change_%': ((row['mean_length'] - baseline['mean_length']) / baseline['mean_length'] * 100),
                    'risk_change_%': ((row['mean_risk'] - baseline['mean_risk']) / baseline['mean_risk'] * 100),
                    'energy_change_%': ((row['mean_energy'] - baseline['mean_energy']) / baseline['mean_energy'] * 100),
                    'turns_change_%': ((row['mean_turns'] - baseline['mean_turns']) / baseline['mean_turns'] * 100),
                    'height_changes_change_%': ((row['mean_height_changes'] - baseline['mean_height_changes']) / baseline['mean_height_changes'] * 100),
                }
                baseline_comparison.append(comparison)
    
    comparison_df = pd.DataFrame(baseline_comparison)
    
    # 3. Summary by likelihood factor
    factor_summary = comparison_df.groupby(['alpha', 'parameter']).agg({
        'risk_change_%': ['mean', 'min', 'max'],
        'length_change_%': ['mean', 'min', 'max'],
        'energy_change_%': ['mean', 'min', 'max'],
        'turns_change_%': ['mean', 'min', 'max'],
        'height_changes_change_%': ['mean', 'min', 'max']
    }).round(2)
    
    # 4. Summary by area type
    area_summary = comparison_df.groupby(['alpha', 'area_type']).agg({
        'risk_change_%': ['mean', 'min', 'max'],
        'length_change_%': ['mean', 'min', 'max'],
        'energy_change_%': ['mean', 'min', 'max'],
        'turns_change_%': ['mean', 'min', 'max'],
        'height_changes_change_%': ['mean', 'min', 'max']
    }).round(2)
    
    return {
        'combined': combined_df_clean,
        'comparison': comparison_df,
        'factor_summary': factor_summary,
        'area_summary': area_summary
    }

In [104]:
# Create summaries
likelihood_summaries = create_likelihood_sa_summary(likelihood_results)

# Get summary by area type for each alpha
for alpha in [0.0, 0.5, 1.0]:
    area_summary = likelihood_summaries['area_summary'].loc[alpha]
    print(f"\nLikelihood Sensitivity Summary α = {alpha}:")
    print(area_summary)


Likelihood Sensitivity Summary α = 0.0:
                              risk_change_%             length_change_%       \
                                       mean   min   max            mean  min   
area_type                                                                      
Meadows and open grass                 0.07 -0.22  0.22             0.0  0.0   
Pedestrian and cycling paths          -0.00 -1.97  1.97             0.0  0.0   
Regional roads                         0.25 -2.51  2.51             0.0  0.0   
Rivers, canals and streams             0.03 -0.10  0.10             0.0  0.0   
Tracks and rural access roads          0.13 -1.29  1.29             0.0  0.0   

                                   energy_change_%           turns_change_%  \
                               max            mean  min  max           mean   
area_type                                                                     
Meadows and open grass         0.0             0.0  0.0  0.0            0.0   
Pe

In [105]:
# Cell 3: Create aggregated summary tables for LaTeX

def create_likelihood_tables_for_latex(summaries):
    """Create formatted tables for likelihood sensitivity analysis."""
    
    comparison_df = summaries['comparison']
    
    # 1. Summary by area type (averaged across all likelihood factors)
    area_summary = comparison_df.groupby(['alpha', 'area_type']).agg({
        'risk_change_%': ['mean', 'std', 'min', 'max'],
        'length_change_%': ['mean', 'std', 'min', 'max'],
        'energy_change_%': ['mean', 'std', 'min', 'max'],
        'turns_change_%': ['mean', 'std', 'min', 'max'],
        'height_changes_change_%': ['mean', 'std', 'min', 'max']
    }).round(2)
    
    # 2. Summary by likelihood factor (averaged across all area types)
    factor_summary = comparison_df.groupby(['alpha', 'parameter']).agg({
        'risk_change_%': ['mean', 'std', 'min', 'max'],
        'length_change_%': ['mean', 'std', 'min', 'max'],
        'energy_change_%': ['mean', 'std', 'min', 'max'],
        'turns_change_%': ['mean', 'std', 'min', 'max'],
        'height_changes_change_%': ['mean', 'std', 'min', 'max']
    }).round(2)
    
    # 3. Create LaTeX-ready tables for each alpha
    latex_tables = {}
    
    for alpha in [0.0, 0.5, 1.0]:
        # Table by area type
        area_alpha = area_summary.loc[alpha]
        area_table = pd.DataFrame({
            'Risk Δ (%)': area_alpha[('risk_change_%', 'mean')].astype(str) + ' ± ' + area_alpha[('risk_change_%', 'std')].astype(str),
            'Risk Range': '(' + area_alpha[('risk_change_%', 'min')].astype(str) + ', ' + area_alpha[('risk_change_%', 'max')].astype(str) + ')',
            'Length Δ (%)': area_alpha[('length_change_%', 'mean')].astype(str) + ' ± ' + area_alpha[('length_change_%', 'std')].astype(str),
            'Energy Δ (%)': area_alpha[('energy_change_%', 'mean')].astype(str) + ' ± ' + area_alpha[('energy_change_%', 'std')].astype(str),
            'Turns Δ (%)': area_alpha[('turns_change_%', 'mean')].astype(str) + ' ± ' + area_alpha[('turns_change_%', 'std')].astype(str),
            'Height Changes Δ (%)': area_alpha[('height_changes_change_%', 'mean')].astype(str) + ' ± ' + area_alpha[('height_changes_change_%', 'std')].astype(str)
        })
        
        # Table by factor
        factor_alpha = factor_summary.loc[alpha]
        factor_table = pd.DataFrame({
            'Risk Δ (%)': factor_alpha[('risk_change_%', 'mean')].astype(str) + ' ± ' + factor_alpha[('risk_change_%', 'std')].astype(str),
            'Risk Range': '(' + factor_alpha[('risk_change_%', 'min')].astype(str) + ', ' + factor_alpha[('risk_change_%', 'max')].astype(str) + ')',
            'Length Δ (%)': factor_alpha[('length_change_%', 'mean')].astype(str) + ' ± ' + factor_alpha[('length_change_%', 'std')].astype(str),
            'Energy Δ (%)': factor_alpha[('energy_change_%', 'mean')].astype(str) + ' ± ' + factor_alpha[('energy_change_%', 'std')].astype(str),
            'Turns Δ (%)': factor_alpha[('turns_change_%', 'mean')].astype(str) + ' ± ' + factor_alpha[('turns_change_%', 'std')].astype(str),
            'Height Changes Δ (%)': factor_alpha[('height_changes_change_%', 'mean')].astype(str) + ' ± ' + factor_alpha[('height_changes_change_%', 'std')].astype(str)
        })
        
        latex_tables[f'alpha_{alpha}_area'] = area_table
        latex_tables[f'alpha_{alpha}_factor'] = factor_table
    
    # 4. Create a detailed view showing which factor affects which area most
    max_effects = comparison_df.loc[comparison_df.groupby(['alpha', 'area_type', 'parameter'])['risk_change_%'].apply(lambda x: x.abs().idxmax())]
    
    return {
        'area_summary': area_summary,
        'factor_summary': factor_summary,
        'latex_tables': latex_tables,
        'max_effects': max_effects,
        'raw_comparison': comparison_df
    }


In [106]:
df_likelihood_alpha0

,alpha,scenario,area_type,parameter,change_direction,length_change_%,risk_change_%,energy_change_%,turns_change_%,height_changes_change_%
4,0.0,Meadows and open grass - Communication -1,Meadows and open grass,Communication,-1,0.0,-0.221815,0.0,0.0,0.0
5,0.0,Meadows and open grass - Communication +1,Meadows and open grass,Communication,1,0.0,0.221815,0.0,0.0,0.0
8,0.0,Meadows and open grass - Electrical -1,Meadows and open grass,Electrical,-1,0.0,0.000000,0.0,0.0,0.0
9,0.0,Meadows and open grass - Electrical +1,Meadows and open grass,Electrical,1,0.0,0.221815,0.0,0.0,0.0
2,0.0,Meadows and open grass - Interference -1,Meadows and open grass,Interference,-1,0.0,0.000000,0.0,0.0,0.0
3,0.0,Meadows and open grass - Interference +1,Meadows and open grass,Interference,1,0.0,0.221815,0.0,0.0,0.0
6,0.0,Meadows and open grass - Navigational -1,Meadows and open grass,Navigational,-1,0.0,0.000000,0.0,0.0,0.0
7,0.0,Meadows and open grass - Navigational +1,Meadows and open grass,Navigational,1,0.0,0.221815,0.0,0.0,0.0
0,0.0,Meadows and open grass - Obstacles -1,Meadows and open grass,Obstacles,-1,0.0,-0.221815,0.0,0.0,0.0
1,0.0,Meadows and open grass - Obstacles +1,Meadows and open grass,Obstacles,1,0.0,0.221815,0.0,0.0,0.0


In [107]:
# Generate the tables
likelihood_tables = create_likelihood_tables_for_latex(likelihood_summaries)

# For α = 0.0
df_likelihood_alpha0 = likelihood_tables['raw_comparison'][likelihood_tables['raw_comparison']['alpha'] == 0.0].copy()
df_likelihood_alpha0 = df_likelihood_alpha0.sort_values(['area_type', 'parameter'])
df_likelihood_alpha0_summary = df_likelihood_alpha0.groupby('area_type').agg({
    'length_change_%': 'mean',
    'risk_change_%': 'mean',
    'energy_change_%': 'mean',
    'turns_change_%': 'mean',
    'height_changes_change_%': 'mean'
}).round(2)

# For α = 0.5
df_likelihood_alpha05 = likelihood_tables['raw_comparison'][likelihood_tables['raw_comparison']['alpha'] == 0.5].copy()
df_likelihood_alpha05 = df_likelihood_alpha05.sort_values(['area_type', 'parameter'])
df_likelihood_alpha05_summary = df_likelihood_alpha05.groupby('area_type').agg({
    'length_change_%': 'mean',
    'risk_change_%': 'mean',
    'energy_change_%': 'mean',
    'turns_change_%': 'mean',
    'height_changes_change_%': 'mean'
}).round(2)

# For α = 1.0
df_likelihood_alpha1 = likelihood_tables['raw_comparison'][likelihood_tables['raw_comparison']['alpha'] == 1.0].copy()
df_likelihood_alpha1 = df_likelihood_alpha1.sort_values(['area_type', 'parameter'])
df_likelihood_alpha1_summary = df_likelihood_alpha1.groupby('area_type').agg({
    'length_change_%': 'mean',
    'risk_change_%': 'mean',
    'energy_change_%': 'mean',
    'turns_change_%': 'mean',
    'height_changes_change_%': 'mean'
}).round(2)

In [108]:
likelihood_tables

{'area_summary':                                     risk_change_%                     \
                                              mean   std    min   max   
 alpha area_type                                                        
 0.0   Meadows and open grass                 0.07  0.18  -0.22  0.22   
       Pedestrian and cycling paths          -0.00  2.08  -1.97  1.97   
       Regional roads                         0.25  2.50  -2.51  2.51   
       Rivers, canals and streams             0.03  0.08  -0.10  0.10   
       Tracks and rural access roads          0.13  1.28  -1.29  1.29   
 0.5   Meadows and open grass                 0.26  4.46  -7.63  3.56   
       Pedestrian and cycling paths          -0.04  0.99  -0.98  0.90   
       Regional roads                        -0.21  1.11  -1.47  0.75   
       Rivers, canals and streams             1.19  7.40 -11.48  6.97   
       Tracks and rural access roads          0.16  3.11  -3.30  2.95   
 1.0   Meadows and open grass      

In [109]:
# For α = 0.0
df_likelihood_alpha0_summary = df_likelihood_alpha0.groupby('area_type').agg({
    'length_change_%': 'mean',
    'risk_change_%': 'mean', 
    'energy_change_%': 'mean',
    'turns_change_%': 'mean',
    'height_changes_change_%': 'mean'
}).round(2)

# For α = 0.5
df_likelihood_alpha05_summary = df_likelihood_alpha05.groupby('area_type').agg({
    'length_change_%': 'mean',
    'risk_change_%': 'mean',
    'energy_change_%': 'mean',
    'turns_change_%': 'mean',
    'height_changes_change_%': 'mean'
}).round(2)

# For α = 1.0
df_likelihood_alpha1_summary = df_likelihood_alpha1.groupby('area_type').agg({
    'length_change_%': 'mean',
    'risk_change_%': 'mean',
    'energy_change_%': 'mean',
    'turns_change_%': 'mean',
    'height_changes_change_%': 'mean'
}).round(2)

# Display the summary tables
print("Likelihood Sensitivity Summary α = 0.0:")
print(df_likelihood_alpha0_summary)
print("\nLikelihood Sensitivity Summary α = 0.5:")
print(df_likelihood_alpha05_summary)
print("\nLikelihood Sensitivity Summary α = 1.0:")
print(df_likelihood_alpha1_summary)


Likelihood Sensitivity Summary α = 0.0:
                               length_change_%  risk_change_%  \
area_type                                                       
Meadows and open grass                     0.0           0.07   
Pedestrian and cycling paths               0.0          -0.00   
Regional roads                             0.0           0.25   
Rivers, canals and streams                 0.0           0.03   
Tracks and rural access roads              0.0           0.13   

                               energy_change_%  turns_change_%  \
area_type                                                        
Meadows and open grass                     0.0             0.0   
Pedestrian and cycling paths               0.0             0.0   
Regional roads                             0.0             0.0   
Rivers, canals and streams                 0.0             0.0   
Tracks and rural access roads              0.0             0.0   

                               height_cha

In [110]:
# If you want more detailed tables with all factors shown separately:
def create_detailed_likelihood_table(df, alpha_value):
    """Create a pivot table showing all likelihood factors for each area type."""
    pivot_risk = df.pivot_table(
        values='risk_change_%',
        index='area_type',
        columns='parameter',
        aggfunc='mean'
    ).round(2)
    
    pivot_length = df.pivot_table(
        values='length_change_%',
        index='area_type',
        columns='parameter',
        aggfunc='mean'
    ).round(2)
    
    pivot_energy = df.pivot_table(
        values='energy_change_%',
        index='area_type',
        columns='parameter',
        aggfunc='mean'
    ).round(2)

    pivot_turns = df.pivot_table(
        values='turns_change_%',
        index='area_type',
        columns='parameter',
        aggfunc='mean'
    ).round(2)

    pivot_height = df.pivot_table(
        values='height_changes_change_%',
        index='area_type',
        columns='parameter',
        aggfunc='mean'
    ).round(2)
    
    return {
        'risk': pivot_risk,
        'length': pivot_length,
        'energy': pivot_energy,
        'turns': pivot_turns,
        'height': pivot_height  
    }

# Create detailed tables for each alpha
detailed_alpha0 = create_detailed_likelihood_table(df_likelihood_alpha0, 0.0)
detailed_alpha05 = create_detailed_likelihood_table(df_likelihood_alpha05, 0.5)
detailed_alpha1 = create_detailed_likelihood_table(df_likelihood_alpha1, 1.0)

In [118]:
df_likelihood_alpha0

,alpha,scenario,area_type,parameter,change_direction,length_change_%,risk_change_%,energy_change_%,turns_change_%,height_changes_change_%
4,0.0,Meadows and open grass - Communication -1,Meadows and open grass,Communication,-1,0.0,-0.221815,0.0,0.0,0.0
5,0.0,Meadows and open grass - Communication +1,Meadows and open grass,Communication,1,0.0,0.221815,0.0,0.0,0.0
8,0.0,Meadows and open grass - Electrical -1,Meadows and open grass,Electrical,-1,0.0,0.000000,0.0,0.0,0.0
9,0.0,Meadows and open grass - Electrical +1,Meadows and open grass,Electrical,1,0.0,0.221815,0.0,0.0,0.0
2,0.0,Meadows and open grass - Interference -1,Meadows and open grass,Interference,-1,0.0,0.000000,0.0,0.0,0.0
3,0.0,Meadows and open grass - Interference +1,Meadows and open grass,Interference,1,0.0,0.221815,0.0,0.0,0.0
6,0.0,Meadows and open grass - Navigational -1,Meadows and open grass,Navigational,-1,0.0,0.000000,0.0,0.0,0.0
7,0.0,Meadows and open grass - Navigational +1,Meadows and open grass,Navigational,1,0.0,0.221815,0.0,0.0,0.0
0,0.0,Meadows and open grass - Obstacles -1,Meadows and open grass,Obstacles,-1,0.0,-0.221815,0.0,0.0,0.0
1,0.0,Meadows and open grass - Obstacles +1,Meadows and open grass,Obstacles,1,0.0,0.221815,0.0,0.0,0.0


In [123]:
# Create summary by area type (averaging across all likelihood factors)
summary_by_area_0 = df_likelihood_alpha0.groupby('area_type').agg({
    'length_change_%': ['mean', 'std', 'min', 'max'],
    'risk_change_%': ['mean', 'std', 'min', 'max'],
    'energy_change_%': ['mean', 'std', 'min', 'max'],
    'turns_change_%': ['mean', 'std', 'min', 'max'],
    'height_changes_change_%': ['mean', 'std', 'min', 'max']
}).round(2)

# Create a cleaner version for the main text
clean_summary_0 = pd.DataFrame({
    'Area Type': summary_by_area_0.index,
    'Risk Δ (%)': summary_by_area_0[('risk_change_%', 'mean')].values,
    'Length Δ (%)': summary_by_area_0[('length_change_%', 'mean')].values,
    'Energy Δ (%)': summary_by_area_0[('energy_change_%', 'mean')].values,
    'Turns Δ (%)': summary_by_area_0[('turns_change_%', 'mean')].values,
    'Alt. Changes Δ (%)': summary_by_area_0[('height_changes_change_%', 'mean')].values
})

# Create summary by area type (averaging across all likelihood factors)
summary_by_area_05 = df_likelihood_alpha05.groupby('area_type').agg({
    'length_change_%': ['mean', 'std', 'min', 'max'],
    'risk_change_%': ['mean', 'std', 'min', 'max'],
    'energy_change_%': ['mean', 'std', 'min', 'max'],
    'turns_change_%': ['mean', 'std', 'min', 'max'],
    'height_changes_change_%': ['mean', 'std', 'min', 'max']
}).round(2)

# Create a cleaner version for the main text
clean_summary_05 = pd.DataFrame({
    'Area Type': summary_by_area_05.index,
    'Risk Δ (%)': summary_by_area_05[('risk_change_%', 'mean')].values,
    'Length Δ (%)': summary_by_area_05[('length_change_%', 'mean')].values,
    'Energy Δ (%)': summary_by_area_05[('energy_change_%', 'mean')].values,
    'Turns Δ (%)': summary_by_area_05[('turns_change_%', 'mean')].values,
    'Alt. Changes Δ (%)': summary_by_area_05[('height_changes_change_%', 'mean')].values
})

# Create summary by area type (averaging across all likelihood factors)
summary_by_area_1 = df_likelihood_alpha1.groupby('area_type').agg({
    'length_change_%': ['mean', 'std', 'min', 'max'],
    'risk_change_%': ['mean', 'std', 'min', 'max'],
    'energy_change_%': ['mean', 'std', 'min', 'max'],
    'turns_change_%': ['mean', 'std', 'min', 'max'],
    'height_changes_change_%': ['mean', 'std', 'min', 'max']
}).round(2)

# Create a cleaner version for the main text
clean_summary_1 = pd.DataFrame({
    'Area Type': summary_by_area_1.index,
    'Risk Δ (%)': summary_by_area_1[('risk_change_%', 'mean')].values,
    'Length Δ (%)': summary_by_area_1[('length_change_%', 'mean')].values,
    'Energy Δ (%)': summary_by_area_1[('energy_change_%', 'mean')].values,
    'Turns Δ (%)': summary_by_area_1[('turns_change_%', 'mean')].values,
    'Alt. Changes Δ (%)': summary_by_area_1[('height_changes_change_%', 'mean')].values
})

In [129]:
summary_by_area_0.to_csv('/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/likelihood_summary_sensitivity_alpha_0.csv', index=False)
summary_by_area_05.to_csv('/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/likelihood_summary_sensitivity_alpha_05.csv', index=False)
summary_by_area_1.to_csv('/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/likelihood_summary_sensitivity_alpha_1.csv', index=False)

In [127]:
summary_by_area_05

length_change_%                   risk_change_%  \
                                         mean   std   min   max          mean   
area_type                                                                       
Meadows and open grass                   0.75  0.78  0.00  2.12          0.26   
Pedestrian and cycling paths             0.08  0.18 -0.09  0.24         -0.04   
Regional roads                           0.67  0.51  0.00  1.14         -0.21   
Rivers, canals and streams               0.55  1.29 -0.09  3.00          1.19   
Tracks and rural access roads            0.26  0.20  0.00  0.49          0.16   

                                                 energy_change_%              \
                                std    min   max            mean   std   min   
area_type                                                                      
Meadows and open grass         4.46  -7.63  3.56            0.71  0.76  0.00   
Pedestrian and cycling paths   0.99  -0.98  0.90            0.04  0.20 -0.15   
Regional roads                 1.11  -1.47  0.75            0.65  0.51  0.00   
Rivers, canals and streams     7.40 -11.48  6.97            0.55  1.25 -0.06   
Tracks and rural access roads  3.11  -3.30  2.95            0.25  0.14  0.00   

                                    turns_change_%                    \
                                max           mean   std   min   max   
area_type                                                              
Meadows and open grass         2.06           1.85  2.62  0.00  6.75   
Pedestrian and cycling paths   0.23          -0.22  0.32 -0.53  0.09   
Regional roads                 1.13           0.91  0.88  0.00  1.75   
Rivers, canals and streams     2.91           3.32  2.60  0.00  5.62   
Tracks and rural access roads  0.40           0.21  0.40 -0.12  0.68   

                              height_changes_change_%                    
                                                 mean   std   min   max  
area_type                                                                
Meadows and open grass                           0.31  0.65  0.00  1.53  
Pedestrian and cycling paths                    -0.51  0.54 -1.02  0.00  
Regional roads                                   0.51  0.54  0.00  1.02  
Rivers, canals and streams                       0.56  0.56  0.00  1.53  
Tracks and rural access roads                    0.31  0.77 -0.51  1.02

In [128]:
summary_by_area_1

length_change_%                   risk_change_%  \
                                         mean   std   min   max          mean   
area_type                                                                       
Meadows and open grass                   0.14  4.01 -2.68  7.39          0.30   
Pedestrian and cycling paths            -0.28  0.28 -0.55 -0.02         -0.03   
Regional roads                          -0.15  0.20 -0.38  0.01          0.07   
Rivers, canals and streams              -1.60  2.22 -3.67  1.18          1.53   
Tracks and rural access roads           -0.49  0.53 -0.99  0.02          0.05   

                                                 energy_change_%              \
                                std    min   max            mean   std   min   
area_type                                                                      
Meadows and open grass         4.98  -8.49  4.00            0.13  3.84 -2.58   
Pedestrian and cycling paths   1.20  -1.16  1.11           -0.26  0.27 -0.51   
Regional roads                 1.11  -1.17  1.07           -0.10  0.20 -0.34   
Rivers, canals and streams     7.97 -11.97  7.86           -1.50  2.07 -3.43   
Tracks and rural access roads  2.81  -3.09  2.56            0.74  0.76  0.00   

                                    turns_change_%                     \
                                max           mean   std    min   max   
area_type                                                               
Meadows and open grass         7.07          -2.12  3.44  -5.25  2.55   
Pedestrian and cycling paths  -0.00          -0.18  0.28  -0.44  0.09   
Regional roads                 0.07           0.08  0.18  -0.11  0.25   
Rivers, canals and streams     1.07          -5.00  4.05 -10.96  0.00   
Tracks and rural access roads  1.46          -0.12  0.16  -0.27  0.04   

                              height_changes_change_%                    
                                                 mean   std   min   max  
area_type                                                                
Meadows and open grass                          -0.44  1.21 -1.47  1.47  
Pedestrian and cycling paths                     0.25  0.26  0.00  0.49  
Regional roads                                   0.49  0.52  0.00  0.98  
Rivers, canals and streams                      -0.10  0.21 -0.49  0.00  
Tracks and rural access roads                   -0.25  0.26 -0.49  0.00

In [112]:
detailed_alpha0

{'risk': parameter                      Communication  Electrical  Interference  \
 area_type                                                                
 Meadows and open grass                  0.00        0.11          0.11   
 Pedestrian and cycling paths           -0.00       -0.00         -0.00   
 Regional roads                          1.26        0.00          0.00   
 Rivers, canals and streams              0.00        0.05          0.05   
 Tracks and rural access roads          -0.00       -0.00          0.64   
 
 parameter                      Navigational  Obstacles  
 area_type                                               
 Meadows and open grass                 0.11        0.0  
 Pedestrian and cycling paths          -0.00       -0.0  
 Regional roads                         0.00        0.0  
 Rivers, canals and streams             0.05        0.0  
 Tracks and rural access roads         -0.00       -0.0  ,
 'length': parameter                      Communication  E

In [113]:
df_likelihood_alpha05.to_csv('/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/likelihood_sensitivity_alpha_0.5.csv', index=False)

In [116]:
df_likelihood_alpha1.to_csv('/Users/cmartens/Documents/thesis_cf_martens/sensitivity_analysis/output/likelihood_sensitivity_alpha_1.0.csv', index=False)